In [1]:
import numpy as np
import scipy as sp
from scipy.stats import norm
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
import os
import time

In [2]:
### Solve the asymptotic parameters

from pynverse import inversefunc

def g_1(x):
    return x * norm.cdf(x) + norm.pdf(x)

def g_2(x):
    return (x**2 + 1) * norm.cdf(x) + x * norm.pdf(x)

def g(x):
    return g_2(inversefunc(g_1, x))

def g_1_inv(x):
    return inversefunc(g_1, x)

def sep_eq_sys(theta, args):
    rho, beta0, kappa = theta
    pi, delta, mu, tau = args
    z1 = -rho * mu - beta0 + kappa * tau
    z2 = -rho * mu + beta0 + kappa
    eq1 = g_1(z1) - rho/(2 * pi * mu * delta)
    eq2 = g_1(z2) - rho/(2 * (1-pi) * mu * delta)
    eq3 = pi * delta * g_2(z1) + (1-pi) * delta * g_2(z2) - 1 + rho**2
    return eq1, eq2, eq3

def solve_sep_eq(pi, mu, n=None, d=None, delta=None, tau=1):
    if delta is None:
        delta = n/d
    args = (pi, delta, mu, tau)
    # (rho, beta0, kappa)
    return sp.optimize.fsolve(lambda theta: sep_eq_sys(theta, args=args), np.zeros(3))

def get_tau(beta0, kappa):   # find the optimal tau
    return (kappa - beta0)/(kappa + beta0)

### Settings

$y_i = \pm 1$,
$\quad \mathbf{x}_{i} \,|\, y_i \sim \mathcal{N}( y_i \boldsymbol\mu, \mathbf I_d)$,
$\quad \boldsymbol\mu = \mu \mathbf{e}_1
= (\mu, 0, \cdots, 0)^\top$.

Assume $\mu$ is a fixed constant.

In [3]:
### Simulating data, fitting SVM, calculating accuracy

class InfeasibleError(Exception):
   """ Infeasible Problem """
   pass

def sim_data(n, d, pi, mu, sig1=1, sig0=1):
    """ Generate Gaussian Mixture Data """
    n1 = int(np.ceil(n * pi))
    n0 = n - n1
    mu1 = np.append(mu, np.zeros(d-1))
    mu0 = -mu1
    Sig1 = sig1*np.identity(d)
    Sig0 = sig0*np.identity(d)
    X1 = np.random.multivariate_normal(mean=mu1, cov=Sig1, size=n1)
    X0 = np.random.multivariate_normal(mean=mu0, cov=Sig0, size=n0)
    X = np.vstack([X1, X0])
    y = np.repeat([1, 0], [n1, n0])
    return [X, y]

def SVM(X, y, C=1, shrinking=True, tol=1e-8, class_weight=None, standard=True):
    clf = SVC(C=C, kernel='linear', shrinking=shrinking, tol=tol, class_weight=class_weight)
    clf.fit(X, y)
    beta = clf.coef_[0]
    beta0 = clf.intercept_[0]
    kappa = 1
    if standard:
        beta_norm = np.linalg.norm(beta)
        beta, beta0, kappa = beta/beta_norm, beta0/beta_norm, kappa/beta_norm
    return [clf, beta, beta0, kappa]

def get_accuracy_01(y_pred, y):
    ind1 = np.where(y == 1)
    ind0 = np.where(y == 0)
    res = y - y_pred
    # calculate overall, majority, minority, accuracy
    acc = np.mean(res == 0)
    acc0 = np.mean(res[ind0] == 0)
    acc1 = np.mean(res[ind1] == 0)
    return [acc, acc0, acc1]

### Calculating calibration

def sigmoid(x):
    if isinstance(x, np.ndarray):
        result = np.where(x > 0, 1/(1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
        return result
    else:
        if x > 0:
            return 1/(1 + np.exp(-x))
        else:
            return np.exp(x)/(1 + np.exp(x))

def get_cal(X, y, beta, beta0, pi, mu):
    ind1 = np.where(y == 1)
    ind0 = np.where(y == 0)
    p_conf = sigmoid(X @ beta + beta0)
    p_Bays = sigmoid(2*X[:,0]*mu + np.log(pi/(1-pi)))
    p_post = sigmoid(2*beta[0]*mu*(X @ beta) + np.log(pi/(1-pi)))
    MSE_raw = (p_conf - y)**2
    CalErr_raw = (p_conf - p_post)**2
    ConfErr_raw = (p_conf - p_Bays)**2
    MSE = pi*np.mean(MSE_raw[ind1]) + (1-pi)*np.mean(MSE_raw[ind0])
    CalErr = pi*np.mean(CalErr_raw[ind1]) + (1-pi)*np.mean(CalErr_raw[ind0])
    ConfErr = pi*np.mean(ConfErr_raw[ind1]) + (1-pi)*np.mean(ConfErr_raw[ind0])
    return [MSE, CalErr, ConfErr]

### Fitting model, calculating result

def get_model(pi, n, d, mu, terminate=False):
    X_train, y_train = sim_data(n=n, d=d, pi=pi, mu=mu)
    md, beta, beta0, kappa = SVM(X_train, y_train, C=1)
    # Check linear separability
    if np.mean(md.predict(X_train) == y_train) < 1:
        if terminate:
            raise InfeasibleError(f'Training data is not separable for pi = {pi:.3f}.')
        else:
            print(f'Training data is not separable for pi = {pi:.3f}.')
    # two support vectors in each class
    idx_SV = np.array([np.where(y_train[md.support_] == 1)[0][0],
                       np.where(y_train[md.support_] == 0)[0][0]])
    SV2 = md.support_vectors_[idx_SV, :]
    return [md, beta, beta0, kappa, SV2]

## Fix $\delta$, plot against $\pi$

### Fix $\delta$, plot against $\pi$, different lines for $\mu$

In [4]:
### Simulate data and calculate metrics

def sim_pi(pi_arr, mu_arr, n, d, REP=10, n_test=100000, seed=2023, terminate=False):
    df = np.zeros((pi_arr.shape[0], (mu_arr.shape[0]), 2, 9))
    # 'rho', 'beta_0', 'kappa', 'err_b', 'err_0', 'err_1', 'MSE', 'calErr', 'confErr'
    np.random.seed(seed)
    for rep_idx in range(REP):
        print(f'replicate = {rep_idx + 1}')
        for mu_idx, mu in enumerate(mu_arr):
            X_test, y_test = sim_data(n=n_test, d=d, pi=0.5, mu=mu)
            for pi_idx, pi in enumerate(pi_arr):
                md, beta, beta0, kappa, SV2 = get_model(pi=pi, n=n, d=d, mu=mu, terminate=terminate)
                df[pi_idx, mu_idx, :, 0] += beta[0]   # rho (same across all tau)
                df[pi_idx, mu_idx, 0, 1] += beta0   # beta0
                df[pi_idx, mu_idx, 0, 2] += kappa   # kappa
                df[pi_idx, mu_idx, 0, 3:6] += get_accuracy_01(md.predict(X_test), y_test)   # acc
                df[pi_idx, mu_idx, 0, 6:9] += get_cal(X_test, y_test, beta, beta0, pi, mu)
                # margin aware with OPTIMAL tau
                tmp_inner = SV2 @ beta
                par_oracle = solve_sep_eq(pi=pi, mu=mu, n=n, d=d)
                tau = get_tau(beta0=par_oracle[1], kappa=par_oracle[2])
                beta0_tau = -np.inner(np.array([1, tau]), tmp_inner)/(1 + tau)
                yhat_test = (np.sign(X_test @ beta + beta0_tau) + 1)/2
                df[pi_idx, mu_idx, 1, 1] += beta0_tau   # beta0
                df[pi_idx, mu_idx, 1, 2] += kappa*2/(1 + tau)   # kappa
                df[pi_idx, mu_idx, 1, 3:6] += get_accuracy_01(yhat_test, y_test)   # acc
                df[pi_idx, mu_idx, 1, 6:9] += get_cal(X_test, y_test, beta, beta0_tau, pi, mu)
    df = df/REP
    df[:, :, :, 3:6] = 1 - df[:, :, :, 3:6]   # change accuracy to error
    return df

BASE_FOLDER = 'delta[fix]-pi[x]-mu'
if not os.path.exists(BASE_FOLDER):
    os.makedirs(BASE_FOLDER)

Parameter settings:

In [5]:
pi_LIST_SIM = np.linspace(0.01, 0.50, 10)
mu_LIST = np.array([1.25, 1.50, 1.75])
n_SAMPLE_SIZE = 1000
d_DIMENSION = 500

FOLDER = BASE_FOLDER + f'/delta({n_SAMPLE_SIZE/d_DIMENSION})-pi-mu({mu_LIST})'
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)

In [6]:
# run and save simulation

# start_time = time.time()
# df = sim_pi(pi_arr=pi_LIST_SIM, mu_arr=mu_LIST, n=n_SAMPLE_SIZE, d=d_DIMENSION,
#             REP=100, n_test=10000, seed=2023, terminate=False)
# end_time = time.time()
# print(f"running time: {end_time - start_time:.6f} seconds")   # 45-50 minutes for 100 replications (REP)
# np.save(f'{FOLDER}/df.npy', df)

In [7]:
# load simulated data

df = np.load(f'{FOLDER}/df.npy')

In [8]:
# record parameters

import json
param = {
    'n': n_SAMPLE_SIZE,
    'd': d_DIMENSION,
    'pi': pi_LIST_SIM.tolist(),
    'mu': mu_LIST.tolist(),
    'REP': 100,
    'n_test': 100000,
    'seed': 2023,
    'terminate': False
}
with open(FOLDER + '/param.json', "w") as file:
    json.dump(param, file)

In [9]:
### Testing and calibration errors in theory

def Err_minor(rho, beta0, mu):
    return norm.cdf(-rho*mu - beta0)

def Err_major(rho, beta0, mu):
    return norm.cdf(-rho*mu + beta0)

def Err_bal(rho, beta0, mu):
    return (norm.cdf(-rho*mu - beta0) + norm.cdf(-rho*mu + beta0))/2

def MeanSquaredErr(rho, beta0, pi, mu):
    rho_mu = rho*mu
    tmp1 = norm.expect(lambda G: (sigmoid(+rho_mu + beta0 + G) - 1)**2)
    tmp2 = norm.expect(lambda G: (sigmoid(-rho_mu + beta0 + G) - 0)**2)
    return pi*tmp1 + (1-pi)*tmp2

def CalibrationErr(rho, beta0, pi, mu):
    rho_mu = rho*mu
    intercept = np.log(pi/(1-pi))
    tmp1 = norm.expect(lambda G: (sigmoid(+rho_mu + beta0 + G) - sigmoid(2*rho_mu*(+rho_mu + G) + intercept))**2)
    tmp2 = norm.expect(lambda G: (sigmoid(-rho_mu + beta0 + G) - sigmoid(2*rho_mu*(-rho_mu + G) + intercept))**2)
    return pi*tmp1 + (1-pi)*tmp2

def ConfidenceErr(rho, beta0, pi, mu):
    mu2 = mu**2
    intercept = np.log(pi/(1-pi))
    tmp1 = norm.expect(lambda G: sigmoid(+2*mu2 + 2*mu*G + intercept)*(1 - sigmoid(+2*mu2 + 2*mu*G + intercept)))
    tmp2 = norm.expect(lambda G: sigmoid(-2*mu2 + 2*mu*G + intercept)*(1 - sigmoid(-2*mu2 + 2*mu*G + intercept)))
    return MeanSquaredErr(rho, beta0, pi, mu) - (pi*tmp1 + (1-pi)*tmp2)

Values of $\pi$ ($x$-axis) used for the plot:

In [10]:
pi_LIST_plot = np.linspace(np.min(pi_LIST_SIM), np.max(pi_LIST_SIM), 100)

In [11]:
def theory_operations(key, rho, beta0, kappa, pi, mu):
    operation = {
        1: lambda rho, beta0, pi, mu: rho,
        2: lambda rho, beta0, pi, mu: beta0,
        3: lambda rho, beta0, pi, mu: kappa,
        4: lambda rho, beta0, pi, mu: Err_bal(rho, beta0, mu),
        5: lambda rho, beta0, pi, mu: Err_major(rho, beta0, mu),
        6: lambda rho, beta0, pi, mu: Err_minor(rho, beta0, mu),
        7: lambda rho, beta0, pi, mu: MeanSquaredErr(rho, beta0, pi, mu),
        8: lambda rho, beta0, pi, mu: CalibrationErr(rho, beta0, pi, mu),
        9: lambda rho, beta0, pi, mu: ConfidenceErr(rho, beta0, pi, mu)
    }
    return operation[key](rho, beta0, pi, mu)

title_list_plain = [
    'Slope',
    'Intercept',
    'Margin',
    'Balanced Error',
    'Majority Error',
    'Minority Error',
    'Mean Squared Error',
    'Calibration Error',
    'Confidence Estimation Error'
]

title_list_latex = [r'$\textbf{' + x + '}$' for x in title_list_plain]

y_label_list = [
    r'$\rho$',
    r'$\beta_0$',
    r'$\kappa$',
    r'$\mathrm{Err}_\mathrm{b}$',
    r'$\mathrm{Err}_-$',
    r'$\mathrm{Err}_+$',
    r'$\mathrm{MSE}$',
    r'$\mathrm{CalErr}$',
    r'$\mathrm{ConfErr}$'
]

#### Single plot

In [12]:
def get_plot(key, sim_data, balanced=True, mu_list=mu_LIST, pi_list_sim=pi_LIST_SIM,
             n_sample_size=n_SAMPLE_SIZE, d_dimension=d_DIMENSION,
             legend=False, fig_size=(4.5, 4), save=False, save_dir = '', latex=False):
    if latex:
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
        title_list = title_list_latex
    else:
        plt.rc('text', usetex=False)
        plt.rc('font', family='sans-serif')
        title_list = title_list_plain
    with plt.style.context('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pacoty.mplstyle'):
        plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#5A5B9F', '#F0C05A', '#FF6F61',
                                                            '#D94F70', '#009473', '#7BC4C4'])
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=fig_size, dpi=100, constrained_layout=True)
        for idx_mu, mu in enumerate(mu_list):
            res_array = np.zeros(pi_LIST_plot.shape[0])
            for idx_pi, pi in enumerate(pi_LIST_plot):
                rho, beta0, kappa = solve_sep_eq(pi=pi, mu=mu, n=n_sample_size, d=d_dimension, tau=1)
                if kappa < 0:
                    print("Non-separable regime")
                if balanced:
                    kappa = beta0 + kappa
                    beta0 = 0
                res_array[idx_pi] = theory_operations(key, rho, beta0, kappa, pi, mu)
            ax.plot(pi_LIST_plot, res_array, label=rf'$\|\boldsymbol{{\mu}}\|_2 = {mu:.2f}$')
            if sim_data is not None:
                ax.scatter(pi_list_sim, sim_data[:, idx_mu, int(balanced), key-1], marker='x')
        ax.set_title(title_list[key-1], fontsize=18)
        ax.set_xlabel(r'$\pi$', fontsize=16)
        ax.set_ylabel(y_label_list[key-1], fontsize=16)
        ax.tick_params(axis='x', labelsize=14)
        ax.tick_params(axis='y', labelsize=14)
        # self-defined legend
        if legend:
            lines = ax.get_lines()
            custom_labels = [
                Line2D([0], [0], color='black', linestyle='-', label='Theory'),
                Line2D([0], [0], color='black', linestyle='', marker='x', label='Simulation')
            ]
            handles, labels = ax.get_legend_handles_labels()
            handles.extend(custom_labels)
            labels.extend([line.get_label() for line in custom_labels])
            def update_prop(handle, orig):
                handle.update_from(orig)
                handle.set_marker('x')
            ax.legend(handles=handles, labels=labels, fontsize=16,
                      handler_map={line: HandlerLine2D(update_func=update_prop) for line in lines})
    if save:
        plt.savefig(save_dir + f'bal={int(balanced)} [{key}] {title_list_plain[key-1]} vs pi, ' +
                    f'n={n_SAMPLE_SIZE}, d={d_DIMENSION}.pdf')
        plt.close()
    else:
        plt.show()
    return None

In [13]:
for i in range(1, 10):
    print(f'Figure {i}')
    get_plot(i, sim_data=df, balanced=False, save=True, legend=i in {4,9} , latex=True, save_dir=FOLDER+'/')

Figure 1
Figure 2
Figure 3
Figure 4
Figure 5
Figure 6
Figure 7
Figure 8
Figure 9


In [14]:
for i in range(1, 10):
    print(f'Figure {i}')
    get_plot(i, sim_data=df, balanced=True, save=True, legend=i in {4,9} , latex=True, save_dir=FOLDER+'/')

Figure 1
Figure 2
Figure 3
Figure 4
Figure 5
Figure 6
Figure 7
Figure 8
Figure 9


#### Merged plot ($\tau = 1$ and $\tau = \tau^\mathrm{opt}$)

In [15]:
from matplotlib.legend_handler import HandlerTuple

def get_plot_comb(key, sim_data, mu_list=mu_LIST, pi_list_sim=pi_LIST_SIM,
                  n_sample_size=n_SAMPLE_SIZE, d_dimension=d_DIMENSION,
                  legend=None, fig_size=(4.5, 4), save=False, save_dir = '', latex=False):
    if latex:
        plt.rc('text', usetex=True)
        plt.rc('font', family='serif')
        plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
        title_list = title_list_latex
    else:
        plt.rc('text', usetex=False)
        plt.rc('font', family='sans-serif')
        title_list = title_list_plain
    with plt.style.context('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pacoty.mplstyle'):
        plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#5A5B9F', '#F0C05A', '#FF6F61',
                                                            '#D94F70', '#009473', '#7BC4C4'])
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=fig_size, dpi=100, constrained_layout=True)
        for idx_mu, mu in enumerate(mu_list):
            res_array = np.zeros((pi_LIST_plot.shape[0], 2))
            col = colors[idx_mu % len(colors)]
            for idx_pi, pi in enumerate(pi_LIST_plot):
                rho, beta0, kappa = solve_sep_eq(pi=pi, mu=mu, n=n_sample_size, d=d_dimension, tau=1)
                if kappa < 0:
                    print("Non-separable regime")
                res_array[idx_pi, 0] = theory_operations(key, rho, beta0, kappa, pi, mu)
                res_array[idx_pi, 1] = theory_operations(key, rho, 0, kappa+beta0, pi, mu)
            ax.plot(pi_LIST_plot, res_array[:, 0], color=col, linestyle='--')
            ax.plot(pi_LIST_plot, res_array[:, 1], color=col)
            if sim_data is not None:
                ax.scatter(pi_list_sim, sim_data[:, idx_mu, 0, key-1], marker='x', color=col)
                ax.scatter(pi_list_sim, sim_data[:, idx_mu, 1, key-1], marker='o', color=col)
        ax.set_title(title_list[key-1], fontsize=18)
        ax.set_xlabel(r'$\pi$', fontsize=16)
        ax.set_ylabel(y_label_list[key-1], fontsize=16)
        ax.tick_params(axis='x', labelsize=14)
        ax.tick_params(axis='y', labelsize=14)
        ax.set_ylim(0, 1)
        # self-defined legend
        if legend == 1:
            legend_list = [None] * 3
            for idx, mu in enumerate(mu_list):
                legend_list[idx] = mpatches.Patch(color=colors[idx % len(colors)],
                                                  label=rf'$\|\boldsymbol{{\mu}}\|_2 = {mu:.2f}$')
            ax.legend(handles=legend_list, fontsize=16)
        elif legend == 2:
            l1 = Line2D([0], [0], label=r'$\tau = 1$', marker='x', linestyle='--', color='k')
            l2 = Line2D([0], [0], label=r'$\tau = \tau^\mathrm{opt}$', marker='o', linestyle='-', color='k')
            ax.legend(handles=[l1, l2], fontsize=16)
        elif legend == 3:
            p1, = ax.plot(np.NaN, np.NaN, 'x', color='k')
            p2, = ax.plot(np.NaN, np.NaN, 'o', color='k')
            l1, = ax.plot(np.NaN, np.NaN, linestyle='--', color='k')
            l2, = ax.plot(np.NaN, np.NaN, linestyle='-', color='k')
            ax.legend([(p1, p2), (l1, l2)], ['Simulation', 'Theory'], fontsize=16,
                      handler_map={tuple: HandlerTuple(ndivide=None)})
    if save:
        plt.savefig(save_dir + f'[{key}] {title_list_plain[key-1]} vs pi, ' +
                    f'n={n_SAMPLE_SIZE}, d={d_DIMENSION}.pdf')
        plt.close()
    else:
        plt.show()
    return None

In [16]:
get_plot_comb(4, sim_data=df, save=True, legend=1, latex=True, save_dir=FOLDER+'/')
get_plot_comb(5, sim_data=df, save=True, legend=2, latex=True, save_dir=FOLDER+'/')
get_plot_comb(6, sim_data=df, save=True, legend=3, latex=True, save_dir=FOLDER+'/')

#### Quick single plot (without simulation, theoretical lines only)

In [17]:
def quick_delta_pi_mu(balanced, mu_list, n_sample_size, d_dimension, save=False, latex=True, legends=None):
    if legends is None:
	    legends = [4,9]
    FOLDER = BASE_FOLDER + f'/delta({n_sample_size/d_dimension})-pi-mu({mu_list})-nosim'
    if not os.path.exists(FOLDER):
        os.makedirs(FOLDER)
    for i in range(1, 10):
        get_plot(i, balanced=balanced, sim_data=None, mu_list=mu_list,
                 n_sample_size=n_sample_size, d_dimension=d_dimension, legend=i in legends,
                 save=save, save_dir=FOLDER + '/', latex=latex)
    return None

In [18]:
quick_delta_pi_mu(balanced=True, mu_list=mu_LIST, n_sample_size=100, d_dimension=20, save=True)

In [19]:
quick_delta_pi_mu(balanced=True, mu_list=2*mu_LIST, n_sample_size=100, d_dimension=100, save=True)

## Fix $\pi, \delta, \mu$, plot against $\tau$

### Fix $\delta, \mu$, plot against $\tau$, different lines for $\pi$

In [20]:
### Simulate data and calculate metrics

def sim_tau(pi, mu, tau_arr, tau_list_plot, n=None, d=None, delta=None, REP=10, n_test=100000, seed=2023, terminate=False,
            legend=None, fig_size=(4.5, 4), save=False, save_dir = '', latex=False):
    inputs = [pi, mu, n, d]
    if_array = []
    for x in inputs:
        try:
            if_array.append(int(len(x) > 1))
        except TypeError:
            if_array.append(0)
    if_array = np.array(if_array)
    if sum(if_array) != 1:
        raise ValueError('Only one of pi, mu, n, d should be an array/list.')
    input_name_LaTeX = [r'\pi', r'\|\boldsymbol{{\mu}}\|_2', r'n', r'd']
    array_id = np.where(if_array == 1)[0][0]   # index of which [pi, mu, n, d] is array
    val_legend_list = np.array(inputs[array_id])
    params = inputs.copy()
    params[array_id] = 0

    if n is not None and d is not None and delta is None:
        df = np.zeros((val_legend_list.shape[0], (tau_arr.shape[0]), 9))
        # 'rho', 'beta_0', 'kappa', 'err_b', 'err_0', 'err_1', 'MSE', 'calErr', 'confErr'
        np.random.seed(seed)
        for rep_idx in range(REP):
            print(f'replicate = {rep_idx + 1}')
            for legend_idx, val_legend in enumerate(val_legend_list):
                params[array_id] = val_legend
                pi, mu, n, d = params
                X_test, y_test = sim_data(n=n_test, d=d, pi=0.5, mu=mu)
                md, beta, beta0, kappa, SV2 = get_model(pi=pi, n=n, d=d, mu=mu, terminate=terminate)

                # margin aware
                tmp_inner = SV2 @ beta
                for tau_idx, tau in enumerate(tau_arr):
                    beta0_tau = -np.inner(np.array([1, tau]), tmp_inner)/(1 + tau)
                    yhat_test = (np.sign(X_test @ beta + beta0_tau) + 1)/2
                    df[legend_idx, tau_idx, 0] += beta[0]   # rho (same across all tau)
                    df[legend_idx, tau_idx, 1] += beta0_tau   # beta0
                    df[legend_idx, tau_idx, 2] += kappa   # kappa
                    df[legend_idx, tau_idx, 3:6] += get_accuracy_01(yhat_test, y_test)   # acc
                    df[legend_idx, tau_idx, 6:9] += get_cal(X_test, y_test, beta, beta0_tau, pi, mu)
        df = df/REP
        df[:, :, 3:6] = 1 - df[:, :, 3:6]   # change accuracy to error
    else:
        df = None

    if legend is None:
        legend = {4, 9}
    with plt.style.context('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pacoty.mplstyle'):
        plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#5A5B9F', '#F0C05A', '#FF6F61',
                                                            '#D94F70', '#009473', '#7BC4C4'])
        for key in range(4, 7):
            print(f'Figure {key}')

            if latex:
                plt.rc('text', usetex=True)
                plt.rc('font', family='serif')
                plt.rc('text.latex', preamble=r'\usepackage{amsmath}')
                title_list = title_list_latex
            else:
                plt.rc('text', usetex=False)
                plt.rc('font', family='sans-serif')
                title_list = title_list_plain

            fig, ax = plt.subplots(nrows=1, ncols=1, figsize=fig_size, dpi=100, constrained_layout=True)
            for legend_idx, val_legend in enumerate(val_legend_list):
                params[array_id] = val_legend
                pi, mu, n, d = params
                if delta is None:
                    delta = n/d
                rho, beta0, kappa = solve_sep_eq(pi=pi, mu=mu, delta=delta, tau=1)
                if kappa < 0:
                        print("Non-separable regime")
                res_array = np.zeros(tau_list_plot.shape[0])
                for idx_tau, tau in enumerate(tau_list_plot):
                    beta0_tau = beta0 + (tau - 1)/(tau + 1)*kappa
                    kappa_tau = kappa + beta0
                    res_array[idx_tau] = theory_operations(key, rho, beta0_tau, kappa_tau, pi, mu)
                ax.plot(tau_list_plot, res_array, label=rf'${input_name_LaTeX[array_id]} = {val_legend:.2f}$')
                if n is not None and d is not None:
                    ax.scatter(tau_arr, df[legend_idx, :, key-1], marker='x')
            ax.set_title(title_list[key-1], fontsize=18)
            ax.set_xlabel(r'$\tau$', fontsize=16)
            ax.set_ylabel(y_label_list[key-1], fontsize=16)
            ax.tick_params(axis='x', labelsize=14)
            ax.tick_params(axis='y', labelsize=14)
            # self-defined legend
            if key in legend:
                lines = ax.get_lines()
                custom_labels = [
                    Line2D([0], [0], color='black', linestyle='-', label='Theory'),
                    Line2D([0], [0], color='black', linestyle='', marker='x', label='Simulation')
                ]
                handles, labels = ax.get_legend_handles_labels()
                handles.extend(custom_labels)
                labels.extend([line.get_label() for line in custom_labels])
                def update_prop(handle, orig):
                    handle.update_from(orig)
                    handle.set_marker('x')
                ax.legend(handles=handles, labels=labels, fontsize=16,
                          handler_map={line: HandlerLine2D(update_func=update_prop) for line in lines})

            if save:
                plt.savefig(save_dir + f'[{key}] {title_list_plain[key-1]} vs tau.pdf')
                plt.close()
            else:
                plt.show()
    return df

BASE_FOLDER = 'delta[fix]-tau[x]-pi'
if not os.path.exists(BASE_FOLDER):
    os.makedirs(BASE_FOLDER)

Parameter settings:

In [21]:
tau_LIST_SIM = np.linspace(1, 6, 10)
pi_LIST = np.array([0.1, 0.15, 0.2])
n_SAMPLE_SIZE = 1000
d_DIMENSION = 500
MU = 2.5

FOLDER = BASE_FOLDER + f'/delta({n_SAMPLE_SIZE/d_DIMENSION})-pi({pi_LIST})-mu({MU})'
if not os.path.exists(FOLDER):
    os.makedirs(FOLDER)

In [22]:
# record parameters

param = {
    'n': n_SAMPLE_SIZE,
    'd': d_DIMENSION,
    'pi': pi_LIST.tolist(),
    'tau': tau_LIST_SIM.tolist(),
    'mu': MU,
    'REP': 100,
    'n_test': 100000,
    'seed': 2024,
    'terminate': False
}
with open(FOLDER + '/param.json', "w") as file:
    json.dump(param, file)

Values of $\tau$ ($x$-axis) used for the plot:

In [23]:
tau_LIST_plot = np.linspace(1, 6, 100)

In [24]:
# run simulation

start_time = time.time()
df = sim_tau(pi=pi_LIST, mu=MU, n=n_SAMPLE_SIZE, d=d_DIMENSION,
             tau_arr=tau_LIST_SIM, tau_list_plot=tau_LIST_plot,
             REP=100, n_test=100000, seed=2024, terminate=False,
             legend={4, 9}, fig_size=(4.5, 4), save=True, save_dir=FOLDER+'/', latex=True)
end_time = time.time()
print(f"running time: {end_time - start_time:.6f} seconds")   # 15-20 minutes for 100 replications (REP)

replicate = 1
replicate = 2
replicate = 3
replicate = 4
replicate = 5
replicate = 6
replicate = 7
replicate = 8
replicate = 9
replicate = 10
replicate = 11
replicate = 12
replicate = 13
replicate = 14
replicate = 15
replicate = 16
replicate = 17
replicate = 18
replicate = 19
replicate = 20
replicate = 21
replicate = 22
replicate = 23
replicate = 24
replicate = 25
replicate = 26
replicate = 27
replicate = 28
replicate = 29
replicate = 30
replicate = 31
replicate = 32
replicate = 33
replicate = 34
replicate = 35
replicate = 36
replicate = 37
replicate = 38
replicate = 39
replicate = 40
replicate = 41
replicate = 42
replicate = 43
replicate = 44
replicate = 45
replicate = 46
replicate = 47
replicate = 48
replicate = 49
replicate = 50
replicate = 51
replicate = 52
replicate = 53
replicate = 54
replicate = 55
replicate = 56
replicate = 57
replicate = 58
replicate = 59
replicate = 60
replicate = 61
replicate = 62
replicate = 63
replicate = 64
replicate = 65
replicate = 66
replicate = 67
repl